In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import cv2
import gc
from tqdm import tqdm
%matplotlib inline

import glob
import os


In [2]:
## No of classes

In [65]:
len(os.listdir('./data/train_simplified'))

340

In [3]:
len(os.listdir('./data/train_raw'))

340

In [4]:
csv_files = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\train_simplified'
split_data = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K'

In [5]:
### I dont have the too much of GPU to process such large dataset 

In [6]:
nrows = 30000
files = os.listdir(csv_files)

In [69]:
## I intensionally commented the for loop since I dont want to execute it again

In [67]:
#for file in tqdm(files, total=340):
    source_file_path = os.path.join(csv, file)
    df = pd.read_csv(source_file_path, usecols = ['countrycode', 'drawing', 'key_id', 'recognized'],
                     nrows = nrows)
    df1 = df.head(int(nrows/2))
    df1_train = df1.head(int(nrows/2/100*90))
    df1_valid = df1.tail(int(nrows/2/100*10))
    
    df2 = df.tail(int(nrows/2))
    df2_train = df2.head(int(nrows/2/100*90))
    df2_valid = df2.tail(int(nrows/2/100*10))
    
    path = os.path.join(split_data,'data_1', 'train')
    os.makedirs(path, exist_ok = True)
    df1_train.to_csv(os.path.join(path, file), index= False)
    path = os.path.join(split_data,'data_1', 'valid')
    os.makedirs(path, exist_ok = True)    
    df1_valid.to_csv(os.path.join(path, file), index= False)
    
    
    path = os.path.join(split_data,'data_2', 'train')
    os.makedirs(path, exist_ok = True)
    df2_train.to_csv(os.path.join(path, file), index= False)
    path = os.path.join(split_data,'data_2', 'valid')
    os.makedirs(path, exist_ok = True)   
    df2_valid.to_csv(os.path.join(path, file), index= False)
    

100%|████████████████████████████████████████████████████████████████████████████████| 340/340 [08:15<00:00,  1.35s/it]


In [81]:
##files = glob.glob(r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K\data_1\train\*.csv')
##len(files)

In [16]:
%matplotlib inline 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import json
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
import cv2 
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy,categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
start = dt.datetime.now()

In [28]:
DP_DIR = './data/shuffle-csvs/'
INPUT_DIR = './data/'

BASE_SIZE = 256
NCSVS = 100
NCATS = 340

np.random.seed(seed=1987)
tf.set_random_seed(seed=1987)

def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    print(files[-1])
    return ([f2cat(f) for f in files])

In [75]:


def apk(actual, predicted, k=3):
    """

    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):

        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

In [76]:
apk(actual= [1,2,3], predicted = [1,2,3])

1.0

In [77]:
def mapk(actual, predicted, k=3):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual,predicted)])

In [78]:
def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

In [79]:
STEPS = 800
EPOCHS = 16
size = 64
batchsize = 680

In [80]:
model = MobileNet(input_shape = (size, size,1), alpha = 1.0, weights = None, classes = NCATS)

model.compile(optimizer = Adam(lr = .002), loss = 'categorical_crossentropy' , 
              metrics = [ categorical_crossentropy,categorical_accuracy,top_3_accuracy])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 65, 65, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 32, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 32, 32, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 32, 32, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 32, 32, 32)        128       
__________

_________________________________________________________________
conv_pw_9_relu (ReLU)        (None, 4, 4, 512)         0         
_________________________________________________________________
conv_dw_10 (DepthwiseConv2D) (None, 4, 4, 512)         4608      
_________________________________________________________________
conv_dw_10_bn (BatchNormaliz (None, 4, 4, 512)         2048      
_________________________________________________________________
conv_dw_10_relu (ReLU)       (None, 4, 4, 512)         0         
_________________________________________________________________
conv_pw_10 (Conv2D)          (None, 4, 4, 512)         262144    
_________________________________________________________________
conv_pw_10_bn (BatchNormaliz (None, 4, 4, 512)         2048      
_________________________________________________________________
conv_pw_10_relu (ReLU)       (None, 4, 4, 512)         0         
_________________________________________________________________
conv_dw_11

Training with Image Generator

In [ ]:
## Drawing Grey Scale Images
def draw_cv2(raw_strokes, size = 64, lw =6, time_color = True):
    img_size = np.zeros((BASE_SIZE,BASE_SIZE), np.uint8)
    for t,stroke in enumerate(raw_strokes):
        for i in range(len(strokes[0]) - 1):
            if time_color:
                color = 255 - min(10,t)*13
            else:
                color = 255
            _ = cv2.line(img, (stroke[0][i],stroke[1][i]),
                        (stroke[0][i+1],stroke[1][i+1]),color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size,size))
    else:
        return img


## Drawing RGB Images
## def draw_cv2_color later on 

In [ ]:
def image_generator_xd(size, batchsize, ks, lw=6, time_color=True):